# Interactive Image Processing with Numba and Bokeh

This demo shows off how interactive image processing can be done in the notebook, using [Numba](http://numba.pydata.org) for numerics, [Bokeh](http://bokeh.pydata.org) for plotting, and Ipython interactors for widgets. The demo runs entirely inside the Ipython notebook, with no Bokeh server required.

Numba must be installed in order to run this demo. To run, click on, `Cell->Run All` in the top menu, then scroll down to individual examples and play around with their controls. 

In [1]:
from __future__ import print_function, division

from timeit import default_timer as timer

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GlyphRenderer, LinearColorMapper
from bokeh.io import push_notebook
from numba import jit, njit

from ipywidgets import interact
import numpy as np
import scipy.misc

In [2]:
output_notebook()

Loading BokehJS ...

## Gaussian Blur

This first section demonstrates performing a simple Gaussian blur on an image. It presents the image, as well as a slider that controls how much blur is applied. Numba is used to compile the python blur kernel, which is invoked when the user modifies the slider. 

*Note:* This simple example does not handle the edge case, so the edge of the image will remain unblurred as the slider is increased. 

In [3]:
# smaller image
img_blur = (scipy.misc.ascent()[::-1,:]/255.0)[:250, :250].copy(order='C')

In [4]:
palette = ['#%02x%02x%02x' %(i,i,i) for i in range(256)]
width, height = img_blur.shape
p_blur = figure(x_range=(0, width), y_range=(0, height))
r_blur = p_blur.image(image=[img_blur], x=[0], y=[0], dw=[width], dh=[height], palette=palette, name='blur')

In [5]:
@njit
def blur(outimg, img, amt):
    iw, ih = img.shape
    for i in range(amt, iw-amt):
        for j in range(amt, ih-amt):
            px = 0.
            for w in range(-amt//2, amt//2):
                for h in range(-amt//2, amt//2):
                    px += img[i+w, j+h]
            outimg[i, j]= px/(amt*amt)

In [6]:
def update(i=0):
    level = 2*i + 1
    
    out = img_blur.copy()
    
    ts = timer()
    blur(out, img_blur, level)
    te = timer()
    print('blur takes:', te - ts)
    
    renderer = p_blur.select(dict(name="blur", type=GlyphRenderer))
    r_blur.data_source.data['image'] = [out]
    push_notebook(handle=t_blur)

In [7]:
t_blur = show(p_blur, notebook_handle=True)

In [8]:
# to kill this, do Kernel/restart

interact(update, i=(0, 10))

A Jupyter Widget

<function __main__.update>